## 0. Library 불러오기 및 경로설정

In [2]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [12]:
import argparse
import multiprocessing
import os
from importlib import import_module

import pandas as pd
import torch
from torch.utils.data import DataLoader

from dataset import TestDataset, MaskBaseDataset

from torchinfo import summary
import numpy as np

In [4]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

data_dir = '/opt/ml/input/data/eval'
model_dir = './model/SEnet01'
output_dir = './output'

### <inference.py start>
<!--  -->
<!-- <font size="5">inference.py</font> -->

In [5]:
def load_model(saved_model, num_classes, device):
    # model_cls = model.MySenet50
    model_cls = getattr(import_module("model"), 'MySenet50')
    model = model_cls(
        num_classes=num_classes
    )

    # tarpath = os.path.join(saved_model, 'best.tar.gz')
    # tar = tarfile.open(tarpath, 'r:gz')
    # tar.extractall(path=saved_model)

    best_model_path = os.path.join(saved_model, 'best.pth')
    # print(best_model_path) # ./model/SEnet01/best.pth
    # print()
    model.load_state_dict(torch.load(best_model_path, map_location=device))

    return model

In [6]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
num_classes = MaskBaseDataset.num_classes  # 18
model = load_model(model_dir, num_classes, device).to(device)
model.eval()

img_root = os.path.join(data_dir, 'images')   # '/opt/ml/input/data/eval/images'
info_path = os.path.join(data_dir, 'info.csv')
info = pd.read_csv(info_path)

In [7]:
info

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,0
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,0
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,0
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,0


In [8]:
img_paths = [os.path.join(img_root, img_id) for img_id in info.ImageID]
dataset = TestDataset(img_paths,(128,96))
loader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=64,
    num_workers=multiprocessing.cpu_count() // 2,
    shuffle=False,
    pin_memory=use_cuda,
    drop_last=False
)

In [29]:
a = next(iter(loader))
print(len(a))
print(a[0].shape)
print(type(a))

my_img = a[0]

64
torch.Size([3, 128, 96])
<class 'torch.Tensor'>


In [31]:
my_img.shape

torch.Size([3, 128, 96])

In [13]:
preds = []
with torch.no_grad():
    for index, images in enumerate(loader):
        images = images.to(device)
        pred = model.forward(images)
        print(pred.shape)
        print(pred[0])
        pred = pred.argmax(dim=-1)
        print(pred[0])
        preds.extend(pred.cpu().numpy())
        if index == 0:
            break
len(preds)

torch.Size([64, 18])
tensor([-14.7595,  -8.2747,  -8.8844, -15.5968, -17.6608, -14.9487, -11.9341,
         -5.9835,  -8.6265, -13.5038, -13.1029, -15.9331,  -2.2673,   3.2675,
          0.0394,  -4.0347,  -4.0085,  -3.3657], device='cuda:0')
tensor(13, device='cuda:0')


64

In [10]:
test = info[:5]
# print(test)
os.makedirs(output_dir, exist_ok=True)

save_path = os.path.join(output_dir, f'output.csv')
print(save_path)
info.to_csv(save_path, index = False)

# output_dir = './output'

./output/output.csv


In [11]:
info['ans'] = preds
save_path = os.path.join(output_dir, f'output.csv')
info.to_csv(save_path, )

ValueError: Length of values (64) does not match length of index (12600)

### <inference.py end>
<!--  -->
<!-- <font size="5">inference.py</font> -->

## 1. Model 정의

In [ ]:
class MyModel(nn.Module):
    def __init__(self, num_classes: int = 1000):
        super(MyModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

## 2. Test Dataset 정의

In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

## 3. Inference

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = MyModel(num_classes=18).to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')